In [1]:
from dask.diagnostics import ProgressBar
from dask.distributed import Client
from dask import delayed

import dask.dataframe as dd
import pandas as pd
import numpy as np

import time

In [2]:
client = Client()

In [25]:
from collections import defaultdict

@delayed
def work():
    k = np.random.choice(list('abcd'))
    v = np.random.rand()
    return {k: v}

@delayed
def combine(futures):
    d = defaultdict(int)
    for future in futures:
        for k, v in future.items():
            d[k] += v
    return d

def loop(n: int):
    futures = [work() for _ in range(int(n))]
    return combine(futures)

In [32]:
%%time
y = loop(1e5)

CPU times: user 4 s, sys: 256 ms, total: 4.26 s
Wall time: 4.22 s


In [33]:
%%time
with ProgressBar():
    r = y.compute()

CPU times: user 1min 36s, sys: 956 ms, total: 1min 37s
Wall time: 2min 28s


In [34]:
r

defaultdict(int,
            {'a': 12573.631866013855,
             'b': 12382.34625443595,
             'c': 12525.846875385218,
             'd': 12481.141500530797})